# LSTM Comment Predictor
An alternative, more standard approach to classifying comments as toxic based on:
https://www.kaggle.com/sbongo/for-beginners-tackling-toxic-using-keras

In [1]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

## Data Extraction
Define the six fields in the training data that we wish to train the neural network to predict and extract their training labels, along with the training and test inputs.

In [3]:
label_cols = ['target',
                'severe_toxicity',
                'obscene',
                'threat',
                'insult',
                'identity_attack']
y = train[label_cols].values
trainComments = train["comment_text"]
testComments = test["comment_text"]


## Data Prep
Use the Keras tokenizer to create a string->int dict and convert every word in the training set to its integer representation. Pad all comments to 200 words.

In [4]:
uniqueWords = 20000
commentLen = 200
tok = Tokenizer(num_words=uniqueWords)
tok.fit_on_texts(list(trainComments))
trainComments_tokenized = tok.texts_to_sequences(trainComments)
testComments_tokenized = tok.texts_to_sequences(testComments)
X_t = pad_sequences(trainComments_tokenized, maxlen=commentLen)
X_te = pad_sequences(testComments_tokenized, maxlen=commentLen)


## Neural Network Construction
Build the Keras neural network. Use an embedding layer to project each integer word index to a vector space, which training will adjust so that more closely associated words are closer in space. These vectors serve as input to a long short-term memory RNN layer. This layer represents a neural network with memory which is capable of classifying long sequences of data, such as our embedded comments. We then use a pooling layer to reduce the LSTM's 3d output to 2D and pass it through a standard neural network, which consists of two dense layers and two 1/10 dropout layers.
We use the sigmoid activation function on the final layer to recieve 6 outputs between 0 and 1, and use the binary_crossentropy loss function, as each of the 6 outputs is a binary classification. 
We create an early-stopping callback so that training can be stopped after a single epoch if loss stopps decreasing.

In [8]:
#input layer for an arbitrary number of comments of length commentLen
inputLayer = Input(shape=(commentLen, ))
#project words to coordinate vector space
x = Embedding(uniqueWords, 128)(inputLayer)
#stateful recurrent neural network
x = LSTM(60, return_sequences = True, name = "lstm")(x)
#reduce 3d vector space output to 2d
x = GlobalMaxPool1D()(x)
#drop 1/10 nodes to improve generalization
x = Dropout(rate=.9)(x)
#standard nn
x = Dense(50, activation='relu')(x)
#drop again
x = Dropout(rate=.9)(x)
#standard nn, sigmoid for values between 0 and 1
x = Dense(6, activation='sigmoid')(x)
model = Model(inputs=inputLayer, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()
earlyStoppingCallback = [EarlyStopping(monitor='val_loss', min_delta=0, mode='min', restore_best_weights=True)]
model2 = model

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 200)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 200, 128)          2560000   
_________________________________________________________________
lstm (LSTM)                  (None, 200, 60)           45360     
_________________________________________________________________
global_max_pooling1d_2 (Glob (None, 60)                0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 60)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                3050      
_________________________________________________________________
dropout_4 (Dropout)          (None, 50)                0         
__________

## NN Training
Train the neural network for 2 epochs using the tokenized comment list and labels. Use 1/10 of the data for validation.

In [7]:
model.fit(X_t, y, batch_size=32, epochs=2, validation_split=.1, callbacks=earlyStoppingCallback)

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 1624386 samples, validate on 180488 samples
Epoch 1/2
 312832/1624386 [====>.........................] - ETA: 1:52:21 - loss: 0.1533 - acc: 0.8525

KeyboardInterrupt: 

In [10]:
preds = model.predict(X_t[:500])

## NN Training (Short)
We can run this to train an NN to about the point where accuracy stops getting much better. In case you're low on time.

In [9]:
model2.fit(X_t[:250000], y[:250000], batch_size=32, epochs=1, validation_split=.1, callbacks=earlyStoppingCallback)
preds = model2.predict(X_t[:500])

Train on 225000 samples, validate on 25000 samples
Epoch 1/1
225000/225000 [==============================] - 1190s 5ms/step - loss: 0.1571 - acc: 0.8572 - val_loss: 0.1204 - val_acc: 0.8532


## Prediction
In this prediction, note the generally higher scores for preds[4] (the insulting comment) and significantly higher scores in preds[4][0] (toxicity score) and preds[4][4] (insult probability) relative to a positive comment.

In [26]:
print(trainComments[0])
print(preds[0])
print(trainComments[4])
print(preds[4])

This is so cool. It's like, 'would you want your mother to read this??' Really great idea, well done!
[0.08750916 0.00224847 0.00889072 0.01100674 0.06541023 0.01494348]
haha you guys are a bunch of losers.
[0.27694565 0.04285502 0.0926517  0.04561752 0.23731375 0.07157156]


Overall, note the difficulty of this challenge - even this method, which scores highly on Kaggle, produces toxicity scores that are higher for toxic comments and lower for non-toxic ones, but don't look a lot like the labels.

In [27]:
print(y[0])
print(y[4])

[0. 0. 0. 0. 0. 0.]
[0.89361702 0.0212766  0.         0.         0.87234043 0.0212766 ]
